In [68]:
import pandas as pd

In [69]:
# Chequear que este en el Drive de los dos integrantes del grupo
file_path_dir = '../sandbox/football/kaggle/'
file_path_appearances = file_path_dir + 'appearances.csv'
file_path_players = file_path_dir + 'players.csv'
file_path_player_valuations = file_path_dir + 'player_valuations.csv'

try:
  appearances = pd.read_csv(file_path_appearances)
  players = pd.read_csv(file_path_players)
  valuations = pd.read_csv(file_path_player_valuations)
  print("CSV loaded successfully.")
except FileNotFoundError:
  print("File not found at the specified path.")
except Exception as e:
  print("An error occurred while loading the CSV:", e)

CSV loaded successfully.


In [70]:
player_appearances = appearances[["player_id", "competition_id"]]
player_valuations_data = valuations[["player_id", "market_value_in_eur", "date"]]
players_with_age = players[["player_id", "name", "date_of_birth"]]

player_valuations_data = player_valuations_data.dropna()
players_with_age = players_with_age.dropna()

player_valuations_data = player_valuations_data.reset_index(drop=True)
players_with_age = players_with_age.reset_index(drop=True)

In [71]:
players_with_age['date_of_birth'] = players_with_age['date_of_birth'].astype('datetime64[ns]')

In [72]:
player_appearances = player_appearances[player_appearances.competition_id.isin(['IT1', 'GB1', 'L1', 'ES1', 'CL'])]
player_appearances = player_appearances.drop_duplicates(subset='player_id', keep='first')
player_appearances

,player_id,competition_id
6836,11680,GB1
6837,13583,GB1
6838,136589,GB1
6839,14007,GB1
6840,14031,GB1
...,...,...
1589500,652331,ES1
1589501,668212,ES1
1589510,939374,ES1
1589605,892217,ES1


In [73]:
player_valuations_data.merge(players_with_age.merge(right=player_appearances, on=['player_id'], how='left').dropna(), on='player_id', how='left').dropna()

,player_id,market_value_in_eur,date,name,date_of_birth,competition_id
1,342216,100000,2001-07-20,Julien Serrano,1998-02-13,CL
3,6893,900000,2003-12-15,Gabriel Tamas,1983-11-09,GB1
4,10,7000000,2004-10-04,Miroslav Klose,1978-06-09,IT1
5,26,1500000,2004-10-04,Roman Weidenfeller,1980-08-06,L1
6,65,8000000,2004-10-04,Dimitar Berbatov,1981-01-30,GB1
...,...,...,...,...,...,...
478174,568005,7000000,2024-07-19,Jeremy Sarmiento,2002-06-16,GB1
478175,661145,5000000,2024-07-19,Jon Aramburu,2002-07-23,ES1
478176,676318,10000000,2024-07-19,Facundo Pellistri,2001-12-20,ES1
478190,23553,600000,2024-08-19,Lennart Thy,1992-02-25,L1


In [74]:
merged_df = players_with_age.merge(right=player_appearances, on='player_id', how='left').dropna()
merged_df = player_valuations_data.merge(merged_df, on='player_id', how='left').dropna()
merged_df['date'] = merged_df['date'].astype('datetime64[ns]')
merged_df.drop(columns=['competition_id'])

,player_id,market_value_in_eur,date,name,date_of_birth
1,342216,100000,2001-07-20,Julien Serrano,1998-02-13
3,6893,900000,2003-12-15,Gabriel Tamas,1983-11-09
4,10,7000000,2004-10-04,Miroslav Klose,1978-06-09
5,26,1500000,2004-10-04,Roman Weidenfeller,1980-08-06
6,65,8000000,2004-10-04,Dimitar Berbatov,1981-01-30
...,...,...,...,...,...
478174,568005,7000000,2024-07-19,Jeremy Sarmiento,2002-06-16
478175,661145,5000000,2024-07-19,Jon Aramburu,2002-07-23
478176,676318,10000000,2024-07-19,Facundo Pellistri,2001-12-20
478190,23553,600000,2024-08-19,Lennart Thy,1992-02-25


In [75]:
def calculate_age(born, date):
    return date.year - born.year - ((date.month, date.day) < (born.month, born.day))

merged_df['age'] = merged_df.apply(lambda x: calculate_age(x['date_of_birth'], x['date']), axis=1)
merged_df = merged_df[merged_df.age > 15]
merged_df = merged_df[['player_id', 'name', 'age', 'market_value_in_eur']]
merged_df

,player_id,name,age,market_value_in_eur
3,6893,Gabriel Tamas,20,900000
4,10,Miroslav Klose,26,7000000
5,26,Roman Weidenfeller,24,1500000
6,65,Dimitar Berbatov,23,8000000
7,77,Lúcio,26,13000000
...,...,...,...,...
478174,568005,Jeremy Sarmiento,22,7000000
478175,661145,Jon Aramburu,21,5000000
478176,676318,Facundo Pellistri,22,10000000
478190,23553,Lennart Thy,32,600000


In [78]:
merged_df.player_id.value_counts()

player_id
39153     52
57162     51
44162     50
35664     50
39381     49
          ..
849865     1
928051     1
928900     1
936536     1
964087     1
Name: count, Length: 8572, dtype: int64

In [76]:
merged_df.to_csv('players_age_valuation.csv', index=False)